##VQE for Quantum Harmonic Oscillator
Estimator primitive is used as VQE function from qiskit_algorithms not working properly

In [ ]:
# Install the required packages
!pip install qiskit[visualization]==1.0.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 68.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.4/39.4 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 17.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.5/107.5 kB 13.5 MB/s eta 0:00:00
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136816 sha256=5dcce63a3bf26ffe8cc79e8ec0507d80ac66b654a84f3c4576af3c1f4678a5c3
  Stored in directory: /root/.cache/pip/wheels/d3/31/8b/e09b0386afd80cfc556c00408c9aeea5c35c4d484a9c762fd5
Successfully built pylatexenc


In [ ]:
!pip install qiskit_aer
!pip install qiskit_ibm_runtime
!pip install matplotlib
!pip install pylatexenc
!pip install prototype-zne
!pip install qiskit_ibm_runtime
!pip install qiskit_nature
!pip install scipy

In [ ]:
#Import the necessary files
from qiskit import QuantumCircuit, QuantumRegister
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit.circuit.library import RealAmplitudes
from qiskit.quantum_info import SparsePauliOp, Pauli
from qiskit_algorithms.optimizers import COBYLA
from qiskit_algorithms import VQE
from qiskit.primitives import Estimator
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import Session, EstimatorV2 as Estimator
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit_nature.second_q.operators import FermionicOp
from qiskit_aer import AerSimulator

In [ ]:
service = QiskitRuntimeService(channel = 'ibm_quantum',
                               token = 'deletethisandpasteyouribmid')

QiskitRuntimeService.save_account(channel='ibm_quantum',
                                  token = 'deletethisandpasteyouribmid')

###Creating the Reference state. This is designed for a single particle but can be scaled to multiparticles. The hamiltonian will change accordingly

In [ ]:
theta = np.pi/3
q = QuantumRegister(1, 'qreg')
qc = QuantumCircuit(q)
qc.h(q[0])
qc.rx(theta, q[0])
qc.draw('mpl')

In [ ]:
# Defining the backend and the ansatz

# backend = service.backend('ibm_brisbane')
backend = AerSimulator()
# Hamiltonian = SparsePauliOp(['I', 'Z'], [0.5, 0.5])
fermionic_op = FermionicOp({'+_0 -_0': 1.0})

jw_mapper = JordanWignerMapper()
# ground_state_solver = GroundStateEigensolver(mapper=hw_mapper)
Hamiltonian = jw_mapper.map(fermionic_op)

#Defining the Ansatz
num_qubits = 1
ansatz = RealAmplitudes(num_qubits, reps = 1, entanglement = None)
ansatz.draw('mpl')

In [ ]:
# Transpilation

optimizationLevel = 1
pm = generate_preset_pass_manager(backend= backend, optimization_level= optimizationLevel)
isa_circuit = pm.run(ansatz)

# isa_circuit.draw('mpl')

In [ ]:
isa_observables = Hamiltonian.apply_layout(isa_circuit.layout)

From here, the estimator function will start

In [ ]:
estimator = Estimator(backend=backend)

optimizer = COBYLA(maxiter = 1000)
vqe = VQE(estimator = estimator, ansatz = ansatz, optimizer = optimizer)

###Cost Function

In [ ]:
def cost_func(params, ansatz, hamiltonian, estimator):
    """Return estimate of energy from estimator

    Parameters:
        params (ndarray): Array of ansatz parameters
        ansatz (QuantumCircuit): Parameterized ansatz circuit
        hamiltonian (SparsePauliOp): Operator representation of Hamiltonian
        estimator (EstimatorV2): Estimator primitive instance
        cost_history_dict: Dictionary for storing intermediate results

    Returns:
        float: Energy estimate
    """
    pub = (ansatz, [hamiltonian], [params])
    h = 6.62607015e-34
    c = 3e8
    w = 2
    result = estimator.run(pubs=[pub]).result()
    energy = h*w*(result[0].data.evs[0] + 0.5)

    cost_history_dict["iters"] += 1
    cost_history_dict["prev_vector"] = params
    cost_history_dict["cost_history"].append(energy)
    print(f"Iters. done: {cost_history_dict['iters']} [Current cost: {energy}]")

    return energy

In [ ]:
cost_history_dict = {
    "prev_vector": None,
    "iters": 0,
    "cost_history": [],
}

In [ ]:
num_params = ansatz.num_parameters
num_params

In [ ]:
x0 = 2 * np.pi * np.random.random(num_params) # Change this later for random parameters

Classical Optimization

In [ ]:
with Session(backend=backend) as session:
    estimator = Estimator(session=session)
    estimator.options.default_shots = 1000

    res = minimize(
        cost_func,
        x0,
        args=(isa_circuit, isa_observables, estimator),
        method="COBYLA",
    )

In [ ]:
print(res) # Printing the Result

Plotting the Curve

In [ ]:
fig, ax = plt.subplots()
ax.plot(range(cost_history_dict["iters"]), cost_history_dict["cost_history"])
ax.set_xlabel("Iterations")
ax.set_ylabel("Cost")
plt.draw()